# Topic Modelling(Clustering)


#### Latent Dirichlet allocation (LDA) is a generative statistical model that allows sets of observations to be explained by unobserved groups that explain why some parts of the data are similar. 

For example, if observations are words collected into documents, it posits that each document is a mixture of a small number of topics and that each word's presence is attributable to one of the document's topics.


#### LDA is an example of a topic model.


LSA-Latent Semantic Analysis   
matrix Factorization(SVD)

In [ ]:
#!pip install gensim 

# it has the package to do Topic modelling 

#### Gensim doesnot take input as Dataframe , but it takes each document with unique terms as a individual list

doc1-[(t1,1),(t3,1)....]  
doc2-[(t1,1),(t3,1),(t5,1)..]  
.
.
doc-n-[(t1,2),(t2,4)...]   

where tuple formed inside the list is(term,frequencyin the doc)  and each term is assigned a unique ID instead of the term in string


In [1]:
# importing the basic libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk
import gensim


In [2]:
# reading the Modi Tweets csv
amazon=pd.read_csv('C:\\Users\\HII\\Desktop\\Text Mining\\10 Text Mining\\amazon_reviews_big.csv')
amazon.head()

,asin,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000HDJXNA,1,What I recieved is not what is pictured here O...,"12 26, 2012",A29YXBFTD7QUP3,HHA,Buyer be ware,1.356480e+09
1,B006KKS7XQ,5,Excellent unit and a pretty simple install usi...,"09 20, 2013",A3IMTXFYD7CGDN,"Peter W. George ""soyflakeman""",high quality without high price,1.379635e+09
2,B002NP8XJ0,5,"I'm enjoying this keyboard, I'm getting anothe...","08 31, 2010",AXNOW20FQKHVW,B. Hayashi,Superb keyboard + solution for slow wake up an...,1.283213e+09
3,B000EITTLE,4,"Overall, this is a fantastic camera that I'm e...","02 3, 2008",A10KCAK279LO0W,"mmcwatters ""macdadi80""",One qualm: not great in low light,1.201997e+09
4,B006CRXK4S,5,These work very well with mySamsung PN64D7000 ...,"01 28, 2012",A19XXLMZXR764J,S. Garfinkle,"Work great, fit well",1.327709e+09


In [3]:
amazon.shape

(100000, 8)



# Bag of word analysis

  We take text column as input and identify the individual words and their frequency of each unique word.
  From those individual words we will come to an idea what area the words are refering .
  
  

In [ ]:
from wordcloud import WordCloud

## Word size /font size depends on the frequency of the word i.e higher freq is in bigger size

In [ ]:

plt.figure(figsize=[16,14])
text=' '.join(map(str,amazon['reviewText'].values))
wc=WordCloud(background_color='white',colormap='plasma').generate(text)
plt.imshow(wc)


# Text Cleaning

In [ ]:
docs=amazon.reviewText.fillna("").str.lower().str.replace('[^a-z ]','')

stopwords=nltk.corpus.stopwords.words('english')
stopwords.extend(["",'use','will','one','good'])# to add custom stopword list to our original list ofn stopwords
stemmer=nltk.stem.PorterStemmer()
# Function to clean the each doc with stopwords and lemmitiation
def clean_doc(doc):
    words=doc.split(" ")
    words_clean=[stemmer.stem(w) for w in words if w not in stopwords]
    words_clean=[w for w in words if w not in stopwords]
    return words_clean
docs_clean=docs.apply(clean_doc)
docs_clean.head()

In [ ]:
# gensim converts the unique word dictionary 
dictionary=gensim.corpora.Dictionary(docs_clean)

In [ ]:
# terms with their Token id
list(zip(dictionary.keys(),dictionary.values()))

In [ ]:
# ID for Word/term Tablet
dictionary.token2id['tablet']

In [ ]:
# Getting the bag of words in each document 
docs_bow=[]
for i in docs_clean:
    bow=dictionary.doc2bow(i)
    docs_bow.append(bow)



# docs_bow has the input format required for gensim package to process it to do Topic classification/ modelling


In [ ]:
lda_mod=gensim.models.LdaMulticore(docs_bow,
                               id2word=dictionary,
                                  num_topics=4,random_state=100,iteration=300)

In [ ]:
                             
#%timeit gensim.models.LdaMulticore(docs_bow,id2word=dictionary,num_topics=4)

In [ ]:
                               
#%timeit gensim.models.LdaModel(docs_bow,id2word=dictionary,num_topics=4)

## Document- Topic Relationship

In [ ]:

lda_mod.get_document_topics(docs_bow[8])


In [ ]:
# unitagging - tagging each doc with highest probable topic
topics=[]
for i in docs_bow:
    df=pd.DataFrame(lda_mod.get_document_topics(i),columns=['topic','Probs'])
    topic=df.sort_values(by='Probs').iloc[-1]['topic']
    topics.append(topic)
 

In [ ]:
amazon['topics']=topics  
amazon.topics.value_counts()

From the above topic list we take out the top 10 high probable words in each topic

In [ ]:
lda_mod.print_topics()